# Portfolio Optimization Problem

We have an opportunity to invest in three different stocks, but our budget is limited. Each stock has its own current price, expected future price, and variance of future price. Our goal is to decide whether and how much we should invest in each stock to minimize the variance (i.e., risk) of total revenue while ensuring the expected revenue is high enough.

Let's define the following variables:

For stock i:
- $p_i$: Current price of stock i.
- $\mu_i$: Expected future price of stock i.
- $\sigma_i^2$: Variance of buying one share of stock i.
- $x_i$: The shares of stock i we buy.

The objective is to minimize the variance of total revenue, which can be formulated as follows:

**Objective Function**:
Minimize $ \sum_{i=1}^{n} \sigma_i^2 x_i^2 $

**Subject to the constraints**:
1. Budget constraint:
   $ \sum_{i=1}^{n} p_i x_i \leq B $
   (The total investment in all stocks should not exceed the budget B).

2. Expected revenue constraint:
   $ \sum_{i=1}^{n} \mu_i x_i \geq R $
   (The total expected revenue should be higher than or equal to the minimum required expected revenue R).

3. Non-negativity constraints:
   $ x_i \geq 0 $ for $ i = 1, \ldots , n $
   (The number of shares bought for each stock should be non-negative).



In [17]:
import pandas as pd
from gurobipy import *

In [18]:
df = pd.read_excel("Portfoilio_optimization .xlsx")

In [19]:
df.reset_index(drop=True, inplace=True)
df.index += 1
df.head()

,Stock,Price,Expected price,Variance of the price
1,1,50,55,100
2,2,40,50,1600
3,3,25,20,100


In [20]:
stock = list(df['Stock'])
price = df['Price']
expected_price = df['Expected price']
variance = df['Variance of the price']

print(stock)

[1, 2, 3]


In [21]:
df2 =pd.read_excel("Portfoilio_optimization .xlsx",sheet_name="Budget and min_exp_profit")

In [22]:
df.reset_index(drop=True, inplace=True)
df.index += 1
df2.head()

,Budget,Minimum expected profit
0,100000,115000


In [23]:
budget = int(df2['Budget'])
min_exp_profit = int(df2['Minimum expected profit'])

In [24]:
model = Model("Portfolio optimization")

In [25]:
#set up the variables
x = model.addVars(stock, vtype=GRB.CONTINUOUS)
model.update()

print(x)

{1: <gurobi.Var C0>, 2: <gurobi.Var C1>, 3: <gurobi.Var C2>}


In [26]:
#set up the objective function
model.setObjective(quicksum((x[i]*x[i])*(variance[i]) for i in stock), GRB.MINIMIZE)

model.update()
model.getObjective()

<gurobi.QuadExpr: 0.0 + [ 100.0 C0 ^ 2 + 1600.0 C1 ^ 2 + 100.0 C2 ^ 2 ]>

In [27]:
#set up the constraints
model.addConstr(quicksum(x[i]*price[i] for i in [1,2,3]) <= budget)
model.addConstr(quicksum(x[i]*expected_price[i] for i in [1,2,3]) >= min_exp_profit)

<gurobi.Constr *Awaiting Model Update*>

In [28]:
model.update()
model.getConstrs()

[<gurobi.Constr R0>, <gurobi.Constr R1>]

In [ ]:
model.write("Portfolio optimization.lp")

In [29]:
model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x84ab935d
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e+01, 6e+01]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+02, 3e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+05, 1e+05]
Presolve time: 0.01s
Presolved: 2 rows, 3 columns, 6 nonzeros
Presolved model has 3 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.03097721e+09 -2.03097721e+09  3.40e+04 2.23e+04  3.

In [30]:
model.write("Portfolio optimization.sol")